In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
GOOGLE_API_KEY="your-gemini-2.5-flash api-key"


<h1>tokenized data are processed</h1>

In [2]:
prompt_policy = """
You are a developer building a vocabulary-teaching game for students. You will be given up to 200 English tokens (single words or a whitespace/comma/newline/semicolon/pipe-separated list). Your job: for each token produce a Bengali rendering (a hint/meaning suitable for the game) and a pedagogical Hardness Score (integer 1-100), and return a single JSON object mapping token → [Bengali rendering, score].
You must process the data any how and retern a json any how, you must and must process the data and retern a json!
MANDATE — what this policy enforces (short)
- MAXIMIZE COVERAGE: After tokenization/segmentation include translations/explanations for **as many tokens as possible**. Aim for **≥95%** coverage. Omit only when token hits the SAFETY FILTER or is non-linguistic garbage.
- HINTS, NOT ANSWERS: Provide Bengali *hints/meanings* that are accurate but **do NOT leak the English word** by using obvious transliterations, acronyms, or English morpheme-strings. For technical items, explain what the thing *is or does* in natural Bengali — **avoid** including substrings or transliterations that directly reveal the English token (e.g., avoid "ডিঅক্সি", "রাইবো", "নিউক্লি", "ডিএনএ", "ফেনাইল", "সাইক্লো" etc. as primary text).
- NO BARE TRANSLITERATION: Do not use phonetic Bengali transliteration as the primary output for semantically-rich tokens. Transliteration is allowed **only** for proper nouns/brands and then only as a short parenthetical adjunct — and only if absolutely necessary.
- JSON ONLY: The model's sole output MUST be a single valid JSON object (UTF-8). No extra text or commentary.

OUTPUT FORMAT (required)
- One JSON object only.
- Keys: the original English token **exactly** as you decide to process (preserve case/punctuation in the key).
- Values: two-element array: [Bengali rendering (string, Bengali Unicode NFC), HardnessScore (integer 1-100)].
- If a token is filtered by SAFETY, omit it silently (do not include nulls/placeholders).

TOKENIZATION & SEGMENTATION (must-follow)
1. Split raw input into candidate tokens on whitespace, commas, semicolons, pipes, tabs, newlines and punctuation (. , ; : ! ? / \\ ( ) [ ] { } " ' `).
2. Split CamelCase, snake_case, kebab-case into sub-tokens.
3. If a candidate token length > 25 characters, attempt segmentation into plausible English words using common prefixes/suffixes/morphemes (e.g., deoxy, ribo, nucleo, -protein, cyclo-, -ene, -amine). Favor segmentations that yield more valid dictionary-like tokens and avoid tiny residues.
4. Treat each resulting segment as an independent JSON key. Do NOT return one concatenated mega-key.

INCLUSION & FALLBACK HIERARCHY (per token)
Try to produce an output for each token using this order:
A. Direct single-word Bengali equivalent (if natural and not revealing).
B. Concise Bengali hint phrase (≤10 Bengali words) that describes category, function, or use — avoid including transliterated morphemes or acronyms that reveal the English token.
C. Decomposition-based Bengali description that explains composition/function **without** reproducing English morphemes in Bengali script. (E.g., say "জৈব প্রোটিন যা জেনেটিক উপাদানের সাথে যুক্ত" instead of "ডিএনএ-সংযুক্ত প্রোটিন".)
D. **LAST RESORT**: transliteration in Bengali script **only** for proper nouns/brands and only as a parenthetical AFTER a Bengali gloss: "<Bengali gloss> (ট্রান্সলিট: <নাম>)". Do NOT use transliteration for scientific/technical compounds as the primary hint.

STRICT TECHNICAL-TERM RULES (chemical/biological/medical/engineering/etc.)
- For any technical token, ALWAYS provide a short Bengali descriptive gloss that conveys class/composition/function in natural Bengali **without** inserting English morpheme transliterations or acronyms as the main clue.
- If morpheme decomposition is helpful, express the meaning in Bengali words explaining components (e.g., "প্রোটিন যা জেনেটিক উপাদানের সঙ্গে যুক্ত" rather than "ডিএনএ-সংযুক্ত প্রোটিন").
- Keep descriptions concise (recommended ≤10 Bengali words). Additional detail may be parenthetical but must still avoid revealing English stems.
- Proper nouns only: allowed transliteration parenthetical (see fallback D).

SAFETY FILTER (non-negotiable)
- Omit tokens that are clearly inappropriate for children (sexually explicit, pornographic, graphic/instructional violent content). Examples: "sex", "vagina", "penis", "porn", "rape", "murder", "torture", "abuse".
- Omit non-linguistic garbage (long runs of punctuation, numeric-only strings, unreadable binary).
- When in doubt about appropriateness, prefer inclusion using a safe Bengali rendering rather than omission.

HARDNESS SCORE (1-100) — concise rubric
- Integer only. No floats.
- Judge each token on an absolute pedagogical scale using these factors: frequency/commonness, length/syllables, morphological complexity, phonetic irregularity, conceptual abstractness, part of speech, familiarity in children’s media.
- Suggested score bands:
  * 1-15: very basic everyday words
  * 16-35: common but slightly advanced
  * 36-60: intermediate/multi-syllabic/abstract
  * 61-80: advanced, rare, or long morphology
  * 81-100: highly technical, specialized, or extremely rare
- Bias scores ~10% downward when uncertain (avoid clustering at 90+).

ENFORCED ANTI-LEAK RULES (do not reveal)
- **MUST NOT** include transliterated English morphemes, acronyms, or obvious English-to-Bengali phonetic fragments as the primary hint (patterns such as ডি, ডি-অক্সি, রাইবো, নিউক্লি, ডিএনএ, সাইক্লো, ফেনাইল, ট্রাইফ্লুরো, হেক্সা etc. are disallowed at start of the hint).
- **If** a candidate Bengali rendering contains as a substring a near-transliteration of the English token (character n-gram overlap), reject it and replace with a Bengali gloss that describes the item without those substrings.
- **Do not** put the English word, parts of it, or obvious transliterations anywhere in the primary gloss. Parenthetical transliteration for proper nouns only.

POST-PRODUCTION QA & SELF-CORRECTION (mandatory)
After producing the JSON, run these checks; if any fail, regenerate/adjust before returning:
1. **Transliteration-detection**: If a rendering begins with or contains obvious transliterated English morphemes or acronyms (see ENFORCED ANTI-LEAK RULES), replace it with a Bengali gloss-first rendering.
2. **Substring match detection**: If the Bengali rendering contains a high-similarity transliteration of the original English token (character n-gram match), replace it.
3. **Coverage check**: If included entries < 95% of token count (after segmentation and filtering), re-run with fallback hierarchy to increase coverage.
4. **Score distribution check**: If >20% of included tokens have score ≥90, re-evaluate scores for spread.
5. **Length check**: If a translation string > ~12 Bengali words, compress to ≤10 words; extra detail may be moved to a short parenthetical that still avoids revealing stems.
6. **Format check**: Ensure every value is an array [string, integer]. If any value violates this, regenerate.

EXAMPLES (required behavioral examples)
- Input: `apple justice zen`
  Output: {"apple": ["একটি সাধারণ ফল", 3], "justice": ["ন্যায় বা ন্যায়বিচারের ধারণা", 10], "zen": ["দর্শনের একটি মনোভাব/ধ্যানমূলক পথ", 45]}
  (Note: hints are natural Bengali phrases that do NOT include English stems.)
- Input: `deoxyribonucleoprotein dicyclopentadienyliron`
  Output: {
    "deoxyribonucleoprotein": ["জিনগত উপাদানের সঙ্গে সংযুক্ত প্রোটিন (আণবিক স্তরে কাজ করা প্রোটিন)", 78],
    "dicyclopentadienyliron": ["সাইক্লোপেন্টাডিয়েন রিংযুক্ত আয়রন-ভিত্তিক যৌগ (আয়রন-ভিত্তিক অর্গানিক যৌগ)", 86]
  }
  (Make sure the primary gloss avoids direct English morpheme transliteration like 'ডিএনএ' or 'ডিক্লো' as the first clue. Use natural Bengali description.)

ADDITIONAL NOTES
- Prefer inclusion: when in doubt about translating a rare or technical token, include it with a short Bengali gloss that doesn't reveal English stems.
- Transliteration is permitted only for proper nouns and only as a parenthetical adjunct — never as the primary hint for technical/lexical tokens.
- STRICTLY return only a single JSON object mapping processed tokens to [BENGALI_HINT, HARDNESS_SCORE].
- Must And Must process the data and retern a json! Must and Must process the data and retern a json! Must and Must process the data and retern a json!
END OF POLICY.
"""


In [3]:
def structured_call(request:str, policy:str = prompt_policy) -> str:
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=GOOGLE_API_KEY)  #type: ignore
    prompt = [SystemMessage(content=policy), HumanMessage(content=request)] #type: ignore
    response = llm.invoke(prompt)   #type: ignore
    return response.content   #type: ignore

In [4]:
import ast
import json
def clearn_and_cheak(response:str):
    try:
            starting_index , last_index = response.index("{") , response.index("}")
            
            data = response[starting_index:last_index+1]
            data = data.replace("'" , '"')
            
            try:
                return json.loads(data)
            except:
                        try:
                            return ast.literal_eval(data)
                        except:
                            print("2")
                            return None  
    except:
        print("1")
        return None


In [5]:
import time

for i in range(60,72+1):
    file_names = f"file_4_token_{i}_.text"
    file_to_save = f"file_4_token_{i}_output.json"

    print(f"Runing for ---------- file_{i}")
   
    with open(file_names , "r") as f:
        words = f.read().strip()
    
    words = words.split(",")
    str_words = ""
    
    for j in words:
        str_words += j.strip() + ","

    while True:
            response = structured_call(str_words)
            cleaned_data = clearn_and_cheak(response)
            if cleaned_data is not None:
                print(cleaned_data)
                break
            else:
                print("Re-Run")
                time.sleep(20)


    with open(file_to_save , "w") as f:
        json.dump(cleaned_data , f , indent=4 , ensure_ascii=False)   #type: ignore
    print("Done")


Runing for ---------- file_60


E0000 00:00:1760076444.533864   13874 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


{'sled': ['বরফের উপর দিয়ে টানার যান', 25], 'slee': ['কাপড় বোনার যন্ত্রাংশ', 65], 'sley': ['কাপড় বোনার যন্ত্রাংশ', 60], 'slew': ['অনেক পরিমাণে; দ্রুত ঘোরানো', 30], 'slid': ['পিছলে গিয়েছিল', 18], 'slik': ['মসৃণ বা চকচকে', 40], 'slim': ['পাতলা বা ছিপছিপে', 15], 'slip': ['পিছলে যাওয়া; ছোট কাগজের টুকরা', 12], 'slit': ['সরু ফাটল বা কাটা', 25], 'slob': ['অলস ও অগোছালো ব্যক্তি', 40], 'slod': ['ধীর গতিতে চলেছিল', 70], 'sloe': ['এক ধরণের ছোট কালো ফল', 55], 'slog': ['কঠোর পরিশ্রম করা; ধীর গতিতে চলা', 35], 'sloo': ['কাদা বা ধীরে চলা', 75], 'slop': ['তরল খাবার বা নোংরা জল', 30], 'slot': ['সরু ছিদ্র বা স্থান', 20], 'slow': ['ধীর গতিতে', 5], 'slub': ['কাপড়ের সুতার ত্রুটি', 65], 'slud': ['কাদা বা পলি', 70], 'slue': ['ঘোরানো বা মোড়ানো', 50], 'slug': ['শামুক জাতীয় প্রাণী; ধাতব টুকরা', 35], 'slum': ['বস্তি এলাকা', 28], 'slur': ['অস্পষ্ট কথা; অপমানজনক মন্তব্য', 40], 'smee': ['এক ধরণের হাঁস', 60], 'smew': ['এক ধরণের ছোট হাঁস', 60], 'smit': ['আঘাত করেছিল', 50], 'smog': ['ধোঁয়া ও কুয়াশার মিশ্রণ', 3

E0000 00:00:1760076507.276827   13874 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


{'sope': ['সাবান বা অনুরূপ পরিষ্কারক', 15], 'soph': ['দ্বিতীয় বর্ষের ছাত্র/ছাত্রী (সংক্ষিপ্ত)', 40], 'sops': ['তরল বা ঝোলে ভেজানো টুকরা; তুষ্ট করার উপকরণ', 35], 'sora': ['এক ধরণের পাখি বা জাপানি নাম', 50], 'sorb': ['শোষণ করা বা ধারণ করা', 60], 'sord': ['নোংরা বা ঘাসযুক্ত জমি (প্রাচীন)', 70], 'sore': ['বেদনাদায়ক; ঘা', 10], 'sori': ['উদ্ভিদের স্পোরাঙ্গিয়া গুচ্ছ', 85], 'sory': ['দুঃখিত বা অনুতপ্ত (প্রাচীন)', 70], 'sorn': ['বিনা পয়সায় অন্যের আশ্রয়ে থাকা (প্রাচীন)', 75], 'sort': ['সাজানো; প্রকার', 10], 'sosh': ['সামাজিক কার্যকলাপ বা পড়াশোনা (কথ্য)', 40], 'soso': ['মোটামুটি; মন্দ নয়', 25], 'soss': ['অলসভাবে বসা; মেশানো (কথ্য)', 70], 'soth': ['সত্য বা বাস্তবতা (প্রাচীন)', 75], 'sots': ['মদ্যপ ব্যক্তিরা', 45], 'soud': ['শব্দ করা বা সাঁতার কাটা (প্রাচীন)', 70], 'souk': ['আরব দেশগুলোর বাজার', 50], 'soul': ['আত্মা; প্রাণ', 20], 'soum': ['চারাভূমির পরিমাপের একক', 80], 'soup': ['তরল খাদ্য', 5], 'sour': ['টক; অম্ল', 10], 'sous': ['অধীনে; সহকারী (ফরাসি)', 40], 'sowf': ['শিস দেওয়া বা দীর্ঘশ্ব

...

{'wych': ['এলম গাছের একটি প্রকার', 60], 'wick': ['মোমবাতির সলতে', 20], 'wide': ['চওড়া', 5], 'wyde': ['চওড়া', 5], 'widu': ['কাঠ', 30], 'wied': ['কাঠ', 50], 'wyes': ['ওয়াই-আকৃতির সংযোগ', 70], 'wife': ['স্ত্রী', 5], 'wigs': ['পরচুলা', 25], 'wyke': ['ছোট খাঁড়ি', 30], 'wild': ['বন্য', 10], 'wile': ['কৌশল', 40], 'wyle': ['কৌশল', 40], 'wily': ['ধূর্ত', 45], 'wilk': ['এক ধরণের সামুদ্রিক শামুক', 65], 'will': ['ইচ্ছা', 5], 'wilt': ['নেতিয়ে পড়া', 35], 'wime': ['ঘোরাঘুরি করা', 70], 'wind': ['বাতাস', 10], 'wynd': ['সরু পথ বা গলি', 60], 'wine': ['মদ', 15], 'wyne': ['মদ', 15], 'wing': ['ডানা', 10], 'winy': ['মদের মতো গন্ধযুক্ত', 50], 'wink': ['চোখ মারা', 20], 'wynn': ['একটি প্রাচীন ইংরেজি অক্ষর', 80], 'wino': ['মদ্যপ ব্যক্তি', 40], 'wins': ['জয় লাভ করে', 10], 'wint': ['গিয়েছিল', 50], 'wipe': ['মোছা', 15], 'wype': ['মোছা', 15], 'wips': ['মোছে', 50], 'wird': ['ভাগ্য', 60], 'wire': ['তার', 15], 'wiry': ['তারের মতো', 40], 'wirl': ['ঘোরা', 60], 'wirr': ['ঘূর্ণন শব্দ', 60], 'wise': ['জ্ঞানী', 10], 

E0000 00:00:1760077111.036273   13874 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


2
Re-Run


E0000 00:00:1760077156.758232   13874 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


{'wugg': ['একটি কাল্পনিক বা অপ্রচলিত শব্দ', 60], 'wulk': ['একটি অপ্রচলিত শব্দ', 65], 'wull': ['একটি অপ্রচলিত বা বিরল শব্দ', 65], 'wush': ['একটি কাল্পনিক বা বিরল শব্দ', 60], 'wusp': ['একটি অপ্রচলিত বা সৃষ্ট শব্দ', 65], 'wuss': ['ভীতু বা দুর্বল ব্যক্তি', 40], 'wust': ['একটি অপ্রচলিত বা অজানা শব্দ', 65], 'wuzu': ['ইসলামে আনুষ্ঠানিক শুদ্ধিকরণ', 70], 'xctl': ['একটি প্রোগ্রামের নিয়ন্ত্রণ নির্দেশিকা', 80], 'xdiv': ['একচেটিয়া বা পারস্পরিক বিভাগ', 75], 'xema': ['এক প্রকার সামুদ্রিক পাখি', 75], 'xeme': ['এক প্রকার বিরল পাখি', 70], 'xiii': ['তেরো (রোমান সংখ্যা)', 25], 'xyla': ['কাঠ সম্পর্কিত একটি উপসর্গ', 70], 'xylo': ['কাঠ বা কাঠের জিনিস সম্পর্কিত', 65], 'xina': ['একটি ব্যক্তিগত নাম', 60], 'xint': ['একটি অজানা বা সংক্ষিপ্ত শব্দ', 70], 'xipe': ['একটি অপ্রচলিত বা কাল্পনিক শব্দ', 65], 'xyst': ['ব্যায়ামের জন্য ব্যবহৃত লম্বা বারান্দা', 80], 'xmas': ['বড়দিন (খ্রিস্টানদের উৎসব)', 15], 'xosa': ['দক্ষিণ আফ্রিকার একটি জাতিগোষ্ঠী', 60], 'xray': ['এক প্রকার উচ্চ-শক্তির বিকিরণ', 35], 'xref': ['অন্যত্র উল